# Downloading Dependences

In [ ]:
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [ ]:
# !apt-get install git-lfs

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased

In [ ]:
# !git lfs install
# !git clone https://huggingface.co/bert-base-multilingual-uncased

In [ ]:
# !pip install transformers==3

# Load Dependences

In [1]:
### add NLP dependences
import pickle
import os
import torch
import pandas as pd
from scipy import stats
import numpy as np

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, train_test_split

from tqdm import tqdm
from collections import OrderedDict, namedtuple
import torch.nn as nn
from torch.optim import lr_scheduler
import joblib

import logging
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup, get_constant_schedule
import sys
from sklearn import metrics, model_selection

import warnings
import torch_xla
import torch_xla.debug.metrics as met
import torch_xla.distributed.data_parallel as dp
import torch_xla.distributed.parallel_loader as pl
import torch_xla.utils.utils as xu
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.test.test_utils as test_utils
import warnings

from torch_xla.core.xla_model import mesh_reduce

warnings.filterwarnings("ignore")

In [2]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions

In [3]:
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [4]:
class BERTBaseUncased(nn.Module):
    def __init__(self, bert_path, output_bert='pooler', NumberOfClasses=2):
        super(BERTBaseUncased, self).__init__()
        self.bert_path = bert_path
        self.bert = transformers.BertModel.from_pretrained(self.bert_path)
        self.bert_drop = nn.Dropout(0.3)
        self.out_last_hidden = nn.Linear(768 * 2, 1)
        self.out_poller = nn.Linear(768, 1)
        self.output_bert = output_bert

        self.NumberOfClasses = NumberOfClasses
        self.out_last_hidden_MultiClasss = nn.Linear(768 * 2, NumberOfClasses)
        self.out_poller_MultiClasss = nn.Linear(768, NumberOfClasses)


    def forward(
            self,
            ids,
            mask,
            token_type_ids
    ):
        o1, o2 = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids)
          
        if self.output_bert=='hidden':
          apool = torch.mean(o1, 1)
          mpool, _ = torch.max(o1, 1)
          cat = torch.cat((apool, mpool), 1)
          bo = self.bert_drop(cat)

          if self.NumberOfClasses > 2:
            output = self.out_last_hidden_MultiClasss(bo)
          else:
            output = self.out_last_hidden(bo)

        else:
          bo = self.bert_drop(o2)

          if self.NumberOfClasses > 2:
            output = self.out_poller_MultiClasss(bo)
 
          else:
            output = self.out_poller(bo)
        
        return output

In [5]:
class BERTDatasetTraining:
    def __init__(self, comment, targets, tokenizer, max_length):
        self.comment = comment
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.targets = targets

    def __len__(self):
        return len(self.comment)

    def __getitem__(self, item):
        comment = str(self.comment[item])
        comment = " ".join(comment.split())

        inputs = self.tokenizer.encode_plus(
            comment,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        padding_length = self.max_length - len(ids)
        
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[item], dtype=torch.float)
        }

In [6]:
#### Handcraft metric for multiclass evaluation
def cem_metric(target, output):
  conf_metrix = metrics.confusion_matrix(target, output)
  cem_metrix = np.zeros(conf_metrix.shape)

  for column in range(conf_metrix.shape[1]):
    for row in range(conf_metrix.shape[0]):
    
      if row == column :
        cem_metrix[row,column] = (conf_metrix.sum(axis=0)[column]/2)/conf_metrix.sum()
                                          
      elif row < column:
        cem_metrix[row,column] = (conf_metrix.sum(axis=0)[column]/2 + conf_metrix.sum(axis=0)[row:column].sum())/conf_metrix.sum()

      elif row > column:
        cem_metrix[row,column] = (conf_metrix.sum(axis=0)[column]/2 + conf_metrix.sum(axis=0)[column+1:row+1].sum())/conf_metrix.sum()

  cem_metrix= - np.log2( np.where(cem_metrix !=0, cem_metrix, cem_metrix+0000000.1 ))

  return np.sum(cem_metrix * conf_metrix.T) / np.sum( np.diag(cem_metrix) * conf_metrix.sum(axis=0))

In [7]:
class TrainModel():
  def __init__(self, PathSaveFiles, BertVersion, BertPath,  OutputBert, LearningRate, BatchSize, Epochs, FileName, X_train, X_valid, y_train ,y_valid, MaxLen = 192, SaveModel=False):
    self.BertVersion = BertVersion
    self.BertPath = BertPath
    self.OutputBert = OutputBert
    self.LearningRate = LearningRate
    self.BatchSize = BatchSize
    self.Epochs = Epochs
    self.FileName = FileName
    self.X_train = X_train
    self.X_valid = X_valid
    self.y_train = y_train
    self.y_valid = y_valid
    self.NumberOfLabels = y_train.nunique()
    self.average_metrics =  'macro' if self.NumberOfLabels > 2 else 'binary'
    self.PathSaveFiles = PathSaveFiles
    self.MaxLen = MaxLen
    self.SaveModel = SaveModel


  def _run(self):
      def OpenEndSave(CurrentEpoch, module):
          if module == 'open'and CurrentEpoch == 1:
            with open(self.PathSaveFiles + self.FileName + ".pkl", "rb") as f:
              self.Results = pickle.load(f)

          elif module == 'save' and CurrentEpoch == (self.Epochs):
            with open(self.PathSaveFiles + self.FileName + ".pkl",'wb') as f:
              pickle.dump(self.Results, f)


      def loss_fn(outputs, targets): 
          if self.NumberOfLabels == 2:
            return nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
          else:
            return nn.CrossEntropyLoss()(outputs, targets.view(-1, 1))
     
            


      def train_loop_fn(data_loader, model, optimizer, device, scheduler=None, epoch=None):
          model.train()
          for bi, d in enumerate(data_loader):
              ids = d["ids"]
              mask = d["mask"]
              token_type_ids = d["token_type_ids"]
              targets = d["targets"]

              ids = ids.to(device, dtype=torch.long)
              mask = mask.to(device, dtype=torch.long)
              token_type_ids = token_type_ids.to(device, dtype=torch.long)
              targets = targets.to(device, dtype=torch.float)
              

              optimizer.zero_grad()
              outputs = model(
                  ids=ids,
                  mask=mask,
                  token_type_ids=token_type_ids
              )

              loss = loss_fn(outputs, targets)
              if bi % 10 == 0:
                  xm.master_print(f'bi={bi}, loss={loss}')

                  ValueLoss = loss.cpu().detach().numpy().tolist()
                  ValueLoss = xm.mesh_reduce('test_loss',ValueLoss, np.mean)
                  self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['loss'].append(ValueLoss)

              loss.backward()
              xm.optimizer_step(optimizer)
              if scheduler is not None:
                  scheduler.step()

      def eval_loop_fn(data_loader, model, device):
          model.eval()
          fin_targets = []
          fin_outputs = []
          for bi, d in enumerate(data_loader):
              ids = d["ids"]
              mask = d["mask"]
              token_type_ids = d["token_type_ids"]
              targets = d["targets"]

              ids = ids.to(device, dtype=torch.long)
              mask = mask.to(device, dtype=torch.long)
              token_type_ids = token_type_ids.to(device, dtype=torch.long)
              targets = targets.to(device, dtype=torch.float)

              outputs = model(
                  ids=ids,
                  mask=mask,
                  token_type_ids=token_type_ids
              )

              targets_np = targets.cpu().detach().numpy().tolist()
              if self.NumberOfLabels == 2:
                outputs_np = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
              else:
                _, outputs = torch.max(outputs, 1)
                outputs_np = outputs.cpu().detach().numpy().tolist()
      
              fin_targets.extend(targets_np)
              fin_outputs.extend(outputs_np)    

          return fin_outputs, fin_targets

      ## Main parameters
      MAX_LEN = self.MaxLen
      tokenizer = transformers.BertTokenizer.from_pretrained(self.BertPath, do_lower_case=True)


      train_dataset = BERTDatasetTraining(
          comment=self.X_train.values,
          targets=self.y_train.values,
          tokenizer=tokenizer,
          max_length=MAX_LEN
      )

      train_sampler = torch.utils.data.distributed.DistributedSampler(
            train_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=True)

      train_data_loader = torch.utils.data.DataLoader(
          train_dataset,
          batch_size=self.BatchSize,
          sampler=train_sampler,
          drop_last=True,
          num_workers=1
      )

      valid_dataset = BERTDatasetTraining(
          comment=self.X_valid.values,
          targets=self.y_valid.values,
          tokenizer=tokenizer,
          max_length=MAX_LEN
      )

      valid_sampler = torch.utils.data.distributed.DistributedSampler(
            valid_dataset,
            num_replicas=xm.xrt_world_size(),
            rank=xm.get_ordinal(),
            shuffle=False)

      valid_data_loader = torch.utils.data.DataLoader(
          valid_dataset,
          batch_size=16,
          sampler=valid_sampler,
          drop_last=False,
          num_workers=1
      )

      device = xm.xla_device()
      model = mx.to(device)
      

      param_optimizer = list(model.named_parameters())
      no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
      optimizer_grouped_parameters = [
          {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
          {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

      
      lr = 0.4 * self.LearningRate * xm.xrt_world_size()
      num_train_steps = int(len(train_dataset) / self.BatchSize / xm.xrt_world_size() * self.Epochs)
      xm.master_print(f'num_train_steps = {num_train_steps}, world_size={xm.xrt_world_size()}')

      optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
      scheduler = get_linear_schedule_with_warmup(
          optimizer,
          num_warmup_steps=0,
          num_training_steps=num_train_steps
      )

      best_f1, f1, best_cem, cem = 0,0,0,0

      for epoch in range(1, self.Epochs+1):
        ## print epoch
          xm.master_print(f'Epoch: {epoch} of {self.Epochs}')
        ## Open file to save results
          OpenEndSave(CurrentEpoch=epoch, module='open')

          para_loader = pl.ParallelLoader(train_data_loader, [device])
          train_loop_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler=scheduler, epoch=epoch)

          para_loader = pl.ParallelLoader(valid_data_loader, [device])
          o, t = eval_loop_fn(para_loader.per_device_loader(device), model, device)
          
          if self.NumberOfLabels == 2:
            outputs = list(np.where(np.array(o) > 0.5, 1, 0))
            f1 = xm.mesh_reduce('validation_f1', metrics.f1_score(t, outputs), np.mean)
            self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['f1'].append(f1)

          else:
            self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['f1_macro'].append(xm.mesh_reduce('validation_f1_macro', metrics.f1_score(t, outputs, average=self.average_metrics), np.mean))
            self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['f1_weighted'].append(cem)
            cem = xm.mesh_reduce('validation_cem', cem_metric(t, outputs), np.mean)
            self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['cem'].append(xm.mesh_reduce('validation_cem', cem_metric(t, outputs), np.mean))

          accuracy = metrics.accuracy_score(t, outputs)
          accuracy = xm.mesh_reduce('test_accuracy', accuracy, np.mean)
          self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['accuracy'].append(accuracy)
          self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['recall'].append(xm.mesh_reduce('validation_recall', metrics.recall_score(t, outputs, average=self.average_metrics), np.mean))
          self.Results[self.BertVersion][self.OutputBert][self.LearningRate][self.BatchSize][epoch]['precision'].append(xm.mesh_reduce('validation_precison', metrics.precision_score(t, outputs, average=self.average_metrics), np.mean))
              
        ## save file with save results
          OpenEndSave(CurrentEpoch=epoch, module='save')

        ## Save model
          if self.SaveModel and epoch == self.Epochs:
            xm.save(model.state_dict(), self.PathSaveFiles + self.FileName + '.bin')
        
        ## print accuracy
          xm.master_print(f'Accuracy = {accuracy}')


#Load data

In [8]:
PathDataSet = "../content/drive/MyDrive/Code/DETOXIS/Data/train.csv"
# Task 1
df_train = pd.read_csv(PathDataSet, usecols=["comment", "toxicity"]).fillna("none")
NewColumnsNames = {"comment":"Data","toxicity":"Label"}

# ## Task 2
# df_train = pd.read_csv(PathDataSet, usecols=["comment", "toxicity_level"]).fillna("none")
# NewColumnsNames = {"comment":"Data","toxicity_level":"Label"}

df_train = df_train.rename(columns=NewColumnsNames)
df_train = df_train.sample(frac=1).reset_index(drop=True)

In [ ]:
# df_train = pd.read_csv("../content/drive/MyDrive/Code/DETOXIS/Data/train.csv", usecols=["comment", "toxicity"]).fillna("none")
# # df_valid = df_train.loc[2770:,['comment','toxicity']].reset_index(drop=True)
# # df_train = df_train.sample(frac=1).reset_index(drop=True)

In [9]:
df_train.head()

,Data,Label
0,he pensado lo mismo al leer el titular,0
1,"Señora Nuria ,aunque supongo será un nombre fi...",1
2,Visto desde ese punto de vista ni tan mal...,1
3,Vaya costumbres que tienen en Extremadura.,0
4,HAHAHAHAHAHAHAHAAHAH\nse me ha escuchado desde...,0


#Load Weights

In [10]:
def CriateFileName(BertVersionDict, NumberOfClasses):
  
  NameFile = str()
  for BertModel in BertVersionDict.keys():
    NameFile += BertModel

  if NumberOfClasses > 2:
    NameFile += 'Task2'
  else:
    NameFile += 'Task1'

  return NameFile

In [ ]:
# BertVersion = {'SpanishBert':'../content/bert-base-spanish-wwm-uncased/', 'MultilingualBert':'../content/bert-base-multilingual-uncased/'}
# OutputBert = ['hidden', 'pooler']
# LearningRate = [1e-5, 3e-5, 5e-5]
# BatchSize = [8, 16, 32 , 64]
# Epochs = 20

In [49]:
BertVersion = {'SpanishBert':'../content/bert-base-spanish-wwm-uncased/'}
OutputBert = ['hidden', 'pooler']
LearningRate = [1e-5, 3e-5, 5e-5]
BatchSize = [8, 16, 32 , 64]
Epochs = 20

In [50]:
## Evalute matrics
###### Task 1
MetricsTask1 = ['accuracy', 'f1', 'recall', 'precision']
###### Task 2
MetricsTask2 = ['accuracy', 'f1_macro', 'f1_weighted', 'recall', 'precision', 'cem']

## Get for 'Binary' classification' task1 or 'Multilabel classifcation' task2
Metrics = MetricsTask2 if df_train['Label'].nunique() > 2 else MetricsTask1

## Criate dictinaril results
ResultsTask = { bert:{ output:{ lr:{ bat:{ epoc:{ metric:[] for metric in Metrics + ['loss']} for epoc in range(1, Epochs+1) } for bat in BatchSize} for lr in LearningRate} for output in OutputBert } for bert in BertVersion.keys() }

In [51]:
## Where to Save Files
Path = 'drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Bert/Results/'

## Creating Main Parte Bert File Name
MainParteBertFileName = CriateFileName(BertVersion, NumberOfClasses=df_train['Label'].nunique())

## Create file to save results
FileResults = MainParteBertFileName + 'Results'
# with open(Path + FileResults + ".pkl",'wb') as f:
#   pickle.dump(ResultsTask, f)

In [ ]:
### Cross Validation
for BertV, BertPath in BertVersion.items():
  for OutputB in OutputBert:

    ### Loading Bert trained weights
    mx = BERTBaseUncased(bert_path=BertPath, output_bert=OutputB, NumberOfClasses=df_train['Label'].nunique())

    for lr in LearningRate:
      for Batch in BatchSize:

        ## StratifiedKFold
        skf = StratifiedKFold(n_splits=10)
        fold = 1
        for train_index, valid_index in skf.split(df_train['Data'], df_train['Label']):
          X_train, X_valid = df_train.loc[train_index, 'Data'], df_train.loc[valid_index, 'Data']
          y_train, y_valid = df_train.loc[train_index, 'Label'], df_train.loc[valid_index, 'Label']

          print(f'parameters: Bertmodel: {BertV}, Output: {OutputB}, lr: {lr}, Batch: {Batch}, Totsl Num. Epochs: {Epochs}, Fold: {fold}')
          fold += 1
          MoDeL = TrainModel(PathSaveFiles = Path,
                            BertVersion=BertV,
                            BertPath=BertPath,
                            OutputBert=OutputB,
                            LearningRate=lr,
                            BatchSize=Batch,
                            Epochs=Epochs,
                            FileName= FileResults,
                            X_train=X_train, 
                            X_valid=X_valid,
                            y_train=y_train,
                            y_valid=y_valid)
        

          def _mp_fn(rank, flags):
            torch.set_default_tensor_type('torch.FloatTensor')
            a = MoDeL._run()

          FLAGS={}
          xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

In [52]:
def AveragResults(FileName, Path):
  with open(Path + FileName + ".pkl", "rb") as f:
              Results = pickle.load(f)

  for BT, ModelBertType,  in Results.items():
    for OP, OutPut in ModelBertType.items():
      for LR, LearningRate in OutPut.items():
        for BS, BatchSize in LearningRate.items():
          for EP, Epoch in BatchSize.items():
            for Metrics, ValuesCrossValidation in  Epoch.items():
 
              # Metrics = np.mean(ValuesCrossValidation)
              Results[BT][OP][LR][BS][EP][Metrics] = np.mean(ValuesCrossValidation)
            
    with open('Mean' + FileName + '.pkl','wb') as f:
      pickle.dump(Results, f)

    with open(Path + 'Mean' + FileName + '.pkl','wb') as f:
      pickle.dump(Results, f)
    
    return Results

In [53]:
## Average and Save Results
AverageResultsTask = AveragResults(FileName=FileResults, Path=Path)

In [54]:
### create dataframe for our results
def create_Data_Frame(all_resultas):

  

  ### Criate a pandas da Frame with all results
  df_results = pd.DataFrame.from_dict({(BertType, OutpuType, LearningRate, BactSize, Epochs): all_resultas[BertType][OutpuType][LearningRate][BactSize][Epochs]
                            for BertType in all_resultas.keys()
                            for OutpuType in all_resultas[BertType].keys()
                            for LearningRate in all_resultas[BertType][OutpuType].keys()
                            for BactSize in all_resultas[BertType][OutpuType][LearningRate].keys()
                            for Epochs in all_resultas[BertType][OutpuType][LearningRate][BactSize].keys()},
                        orient='index')
  return df_results

In [55]:
## Create a Data Frame
DfResultsTask = create_Data_Frame(all_resultas=AverageResultsTask)

### save results to a CSV file
DfResultsTask.to_csv(Path + 'Average' + FileResults + '_CSV_' + '.csv')

### See the Avarage results in the Pandas data Frame
DfResultsTask

accuracy        f1  ...  precision      loss
SpanishBert hidden 0.00001 8  1   0.728125  0.450232  ...   0.721950  0.624056
                              2   0.750852  0.511294  ...   0.739075  0.438842
                              3   0.742045  0.576718  ...   0.664583  0.243240
                              4   0.747159  0.591687  ...   0.654069  0.172412
                              5   0.749148  0.561331  ...   0.713554  0.163945
...                                    ...       ...  ...        ...       ...
            pooler 0.00005 64 16  0.753125  0.615895  ...   0.647149  0.004763
                              17  0.748864  0.608242  ...   0.641783  0.002519
                              18  0.749432  0.607221  ...   0.643004  0.002429
                              19  0.749432  0.607502  ...   0.643796  0.002171
                              20  0.749148  0.607561  ...   0.643114  0.002141

[480 rows x 5 columns]

In [56]:
## Creating LateX Table
LabelTaskTable = FileResults
print(DfResultsTask.to_latex(multicolumn=True, multirow=False, label=LabelTaskTable))

\begin{table}
\centering
\label{SpanishBertTask1Results}
\begin{tabular}{lllllrrrrr}
\toprule
            &        &         &    &    &  accuracy &        f1 &    recall &  precision &      loss \\
\midrule
SpanishBert & hidden & 0.00001 & 8  & 1  &  0.728125 &  0.450232 &  0.354750 &   0.721950 &  0.624056 \\
            &        &         &    & 2  &  0.750852 &  0.511294 &  0.429728 &   0.739075 &  0.438842 \\
            &        &         &    & 3  &  0.742045 &  0.576718 &  0.557383 &   0.664583 &  0.243240 \\
            &        &         &    & 4  &  0.747159 &  0.591687 &  0.585898 &   0.654069 &  0.172412 \\
            &        &         &    & 5  &  0.749148 &  0.561331 &  0.498089 &   0.713554 &  0.163945 \\
            &        &         &    & 6  &  0.743750 &  0.603858 &  0.592132 &   0.651411 &  0.062831 \\
            &        &         &    & 7  &  0.746591 &  0.598161 &  0.580012 &   0.656462 &  0.016762 \\
            &        &         &    & 8  &  0.750000 &  0

# Inference

##Train the model with Full Train dataset

In [57]:
## 10 Best resuts
MetricForBestResults = 'cem' if df_train['Label'].nunique() > 2 else 'f1'
DfResultsTask.nlargest(n=10, columns= MetricForBestResults )

accuracy        f1  ...  precision      loss
SpanishBert pooler 0.00001 32 4   0.744602  0.631424  ...   0.633806  0.292655
                           64 7   0.741477  0.627647  ...   0.618375  0.264552
                           16 7   0.726705  0.626452  ...   0.601620  0.105680
                   0.00005 64 14  0.755398  0.624468  ...   0.648528  0.006428
                   0.00003 64 16  0.756534  0.623729  ...   0.656825  0.003672
                              17  0.755682  0.621099  ...   0.654847  0.003256
            hidden 0.00001 32 5   0.742614  0.619629  ...   0.642867  0.230715
            pooler 0.00003 16 5   0.742898  0.618628  ...   0.627488  0.095520
                   0.00005 16 6   0.738636  0.618375  ...   0.633590  0.043906
                           64 12  0.756534  0.617324  ...   0.657511  0.020571

[10 rows x 5 columns]

In [21]:
## Get best parameters from cross-validation DataFrame 
BestResultParameters = DfResultsTask.sort_values(MetricForBestResults, ascending=False)[:1].index
print(f'Best parameters : {BestResultParameters}')

Best parameters : MultiIndex([('SpanishBert', 'pooler', 1e-05, 32, 4)],
           )


In [22]:
## Add best parameters to variables in the final train
BertPath = BertVersion[BestResultParameters[0][0]]
BertVersion = {BestResultParameters[0][0] : BertVersion[BestResultParameters[0][0]]}
OutputBert = [BestResultParameters[0][1]]
LearningRate = [float(BestResultParameters[0][2])]
BatchSize = [int(BestResultParameters[0][3])]
Epochs = int(BestResultParameters[0][4])

In [23]:
## Criate dictinaril results
ResultsTaskBestParameters = { bert:{ output:{ lr:{ bat:{ epoc:{ metric:[] for metric in Metrics + ['loss']} for epoc in range(1, Epochs+1) } for bat in BatchSize} for lr in LearningRate} for output in OutputBert } for bert in BertVersion.keys() }

## Create file to save results BEST Parameters
#### Create file name
FileResultsBestModel = FileResults + 'BestModel'
#### Save the file fro results BEST Parameters
with open(Path + FileResultsBestModel + ".pkl",'wb') as f:
  pickle.dump(ResultsTaskBestParameters, f)

In [24]:
## Train with Best parameters

## Best parameters
BertV = BestResultParameters[0][0]
BertPath = BertVersion[BestResultParameters[0][0]]
OutputB = OutputBert[0]
lr = LearningRate[0]
Batch = BatchSize[0]
Epochs = Epochs

### Loading Bert trained weights
mx = BERTBaseUncased(bert_path=BertPath, output_bert=OutputB, NumberOfClasses=df_train['Label'].nunique())

## Split train and test
X_train = df_train['Data']
y_train = df_train['Label']
_, X_test, _, y_test = train_test_split(df_train['Data'], df_train['Label'], test_size=0.33, random_state=42)

print(f'parameters: Bertmodel: {BertV}, Output: {OutputB}, lr: {lr}, Batch: {Batch}, Totsl Num. Epochs: {Epochs}')
MoDeL = TrainModel(PathSaveFiles = Path,
                  BertVersion=BertV,
                  BertPath=BertPath,
                  OutputBert=OutputB,
                  LearningRate=lr,
                  BatchSize=Batch,
                  Epochs=Epochs,
                  FileName= FileResultsBestModel,
                  X_train=X_train, 
                  X_valid=X_test,
                  y_train=y_train,
                  y_valid=y_test,
                  SaveModel=True)


def _mp_fn(rank, flags):
  torch.set_default_tensor_type('torch.FloatTensor')
  a = MoDeL._run()

FLAGS={}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method='fork')

Some weights of the model checkpoint at ../content/bert-base-spanish-wwm-uncased/ were not used when initializing BertModel: ['bert.embeddings.position_ids']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../content/bert-base-spanish-wwm-uncased/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


parameters: Bertmodel: SpanishBert, Output: pooler, lr: 1e-05, Batch: 32, Totsl Num. Epochs: 4
num_train_steps = 54, world_size=8
Epoch: 1 of 4
bi=0, loss=0.6444810032844543
bi=10, loss=0.5622158646583557
Accuracy = 0.7062937062937062
Epoch: 2 of 4
bi=0, loss=0.4690134525299072
bi=10, loss=0.4536563754081726
Accuracy = 0.8006993006993006
Epoch: 3 of 4
bi=0, loss=0.3437179923057556
bi=10, loss=0.32595375180244446
Accuracy = 0.8496503496503496
Epoch: 4 of 4
bi=0, loss=0.248634472489357
bi=10, loss=0.32738834619522095
Accuracy = 0.8697552447552447


In [26]:
## Average and Save Results
AverageResultsTaskBestModel = AveragResults(FileName=FileResultsBestModel, Path=Path)

In [27]:
## Create a Data Frame
DfResultsTaskBestModel = create_Data_Frame(all_resultas=AverageResultsTaskBestModel)

### save results to a CSV file
DfResultsTaskBestModel.to_csv(Path + 'Average' + FileResultsBestModel + '_CSV_' + '.csv')

### See the Avarage results in the Pandas data Frame
DfResultsTaskBestModel

accuracy        f1  ...  precision      loss
SpanishBert pooler 0.00001 32 1  0.706294  0.233842  ...   0.833523  0.614177
                              2  0.800699  0.615392  ...   0.843831  0.515099
                              3  0.849650  0.738477  ...   0.864639  0.398094
                              4  0.869755  0.801625  ...   0.804803  0.342449

[4 rows x 5 columns]

## Inference on Test Dataset

## Load data

In [28]:
PathDataSet = "../content/drive/MyDrive/Code/DETOXIS/Data/test.csv"
df_test = pd.read_csv(PathDataSet, usecols=["comment_id","comment"]).fillna("none")
NewColumnsNames = {"comment":"Data"}
df_test = df_test.rename(columns=NewColumnsNames)

In [29]:
class BERTDatasetTest:
    def __init__(self, comment_text, tokenizer, max_length):
        self.comment_text = comment_text
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, item):
        comment_text = str(self.comment_text[item])
        comment_text = " ".join(comment_text.split())

        inputs = self.tokenizer.encode_plus(
            comment_text,
            None,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_length,
        )
        ids = inputs["input_ids"]
        token_type_ids = inputs["token_type_ids"]
        mask = inputs["attention_mask"]
        
        padding_length = self.max_length - len(ids)
        
        ids = ids + ([0] * padding_length)
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

In [30]:
## Bert tozenizer
tokenizer = transformers.BertTokenizer.from_pretrained(BertPath, do_lower_case=True)

In [31]:
## Loading the best model
device = torch.device("xla")
model = BERTBaseUncased(bert_path=BertPath, output_bert=OutputB, NumberOfClasses=df_train['Label'].nunique()).to(device)
FileBestModel = Path + FileResultsBestModel + '.bin'
model.load_state_dict(torch.load(FileBestModel))
model.eval()

Some weights of the model checkpoint at ../content/bert-base-spanish-wwm-uncased/ were not used when initializing BertModel: ['bert.embeddings.position_ids']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../content/bert-base-spanish-wwm-uncased/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTBaseUncased(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [32]:
## Prepresing the data
valid_dataset = BERTDatasetTest(
        comment_text=df_test['Data'].values,
        tokenizer=tokenizer,
        max_length=192
)

valid_data_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=Batch,
    drop_last=False,
    num_workers=4,
    shuffle=False
)

In [33]:
## Making the Inferences
with torch.no_grad():
    fin_outputs = []
    for bi, d in tqdm(enumerate(valid_data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)

        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        outputs_np = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
        # outputs_np = outputs.detach().cpu().numpy().tolist()
        fin_outputs.extend(outputs_np) 

28it [00:12,  2.21it/s]


In [35]:
## List with Results
fin_outputs

## create a Dataframe from List of Results
df_results = pd.DataFrame.from_records(fin_outputs)

# ## get the model inference
# df_results['Inference'] = df_results.idxmax(axis=1)
df_results['Inference'] = df_results[0].apply(lambda x : 1 if x > 0.5 else 0)

## Visualize results
df_results.head()

,0,Inference
0,0.806787,1
1,0.115982,0
2,0.786314,1
3,0.851175,1
4,0.742455,1


In [37]:
## Get rows index
df_idex = df_test.loc[:,"comment_id"].to_frame()

## Add index to the Results dataframe
df_results = df_results.join(df_idex)

### save results to a CSV file
df_results.to_csv(Path + 'ModelInfereneces' + FileResultsBestModel + '_CSV_' + '.csv')

## ## Visualize results
df_results.head()

,0,Inference,comment_id
0,0.806787,1,10_001
1,0.115982,0,10_002
2,0.786314,1,10_003
3,0.851175,1,10_004
4,0.742455,1,10_005


In [38]:
# Change the data to the DETOXIS format submition
### cerate a data frame only with the labels and ids
df_SubmationResults = df_results.loc[:, ['Inference', 'comment_id']]
df_SubmationResults.head()

,Inference,comment_id
0,1,10_001
1,0,10_002
2,1,10_003
3,1,10_004
4,1,10_005


In [39]:
## create a new id column
df_SubmationResults['id'] = np.arange(len(df_SubmationResults))
## removing olde id comment_id column
df_SubmationResults = df_SubmationResults.loc[:,['id', 'Inference']]
#submation format
df_SubmationResults.head()

,id,Inference
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1


In [40]:
## saive inferences as .tsv
Path = 'drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Bert/Results/'
FileName = 'AI-UPV_subtask1_1'
df_SubmationResults.to_csv( Path + FileName + '.tsv', header=False, sep='\t', index=False)

# Reviewing results

In [42]:
df = pd.read_csv(Path + 'Submited_' + FileName + '.tsv', header=None, sep='\t')
df = df.rename({0:'id', 1:'Inference'}, axis=1)
df.head()

,id,Inference
0,0,0
1,1,0
2,2,1
3,3,1
4,4,1


In [45]:
df['Submited_Inference'] = df_SubmationResults['Inference']
df.head()

,id,Inference,Submited_Inference
0,0,0,1
1,1,0,0
2,2,1,1
3,3,1,1
4,4,1,1


In [46]:
df['check'] = df.apply(lambda x: 1 if x.Inference != x.Submited_Inference else 0, axis=1)
df.head()

,id,Inference,Submited_Inference,check
0,0,0,1,1
1,1,0,0,0
2,2,1,1,0
3,3,1,1,0
4,4,1,1,0


In [47]:
df.check.sum()

62

# Util when the process stops sandly

In [17]:
# import pickle
# path = 
# File = 
#  with open(path + File + ".pkl", "rb") as f:
#    Re = pickle.load(f)
# Re

In [ ]:
# def CleanBrokeTrain(FileName, Path, NumberOfFoldes=10):
#   with open(Path + FileName + ".pkl", "rb") as f:
#               Results = pickle.load(f)

#   for BT, ModelBertType,  in Results.items():
#     for OP, OutPut in ModelBertType.items():
#       for LR, LearningRate in OutPut.items():
#         for BS, BatchSize in LearningRate.items():
#           for EP, Epoch in BatchSize.items():
#             for Metrics, ValuesCrossValidation in  Epoch.items():
 
#               if len(ValuesCrossValidation) != 0 and not len(ValuesCrossValidation) == NumberOfFoldes:
#                 Results[BT][OP][LR][BS][EP][Metrics] = []
            
#   with open(FileName + '.pkl','wb') as f:
#     pickle.dump(Results, f)

#   with open(Path + FileName + '.pkl','wb') as f:
#     pickle.dump(Results, f)

In [ ]:
# CleanBrokeTrain(FileName=FileResults, Path=Path, NumberOfFoldes=10)

In [ ]:
# import pickle
# with open('drive/MyDrive/Code/DETOXIS/Machine-Learning-Tweets-Classification/Bert/Results/SpanishBertTask2Results' + ".pkl", "rb") as f:
#   RE = pickle.load(f)
# RE